## validate checkpoint

In [ ]:
import subprocess

command = [
    "python3",
     "tests/validate_dataset_checkpoint.py"
]
subprocess.run(command)

## test run on updated config

In [ ]:
import subprocess

config_file = "audioldm_train/config/audioldm_original_with_imagebind/audioldm_with_imagebind.yaml"
command = [
    "python3", 
    "audioldm_train/train/latent_diffusion.py", 
    "-c", config_file,
]

# Run the command
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Print output in real-time
for line in process.stdout:
    print(line, end="")

# Wait for the process to finish
process.wait()

KeyboardInterrupt: 

## testing the imagebind model

In [ ]:
from imagebind.imagebind import data
import torch
from imagebind.imagebind.models import imagebind_model
from imagebind.imagebind.models.imagebind_model import ModalityType

# text_list=["A dog.", "A car", "A bird"]
video_paths=["data/dataset/videoset/scratch/shared/beegfs/hchen/train_data/VGGSound_final/video/-__L3T1Yv_4_000000.mp4",
             "data/dataset/videoset/scratch/shared/beegfs/hchen/train_data/VGGSound_final/video/-_-SejUs_VI_000063.mp4"]
# audio_paths=[".assets/dog_audio.wav", ".assets/car_audio.wav", ".assets/bird_audio.wav"]

device = "cuda" if torch.cuda.is_available() else "cpu"

# Instantiate model
model = imagebind_model.imagebind_huge(pretrained=True)
model.eval()
model.to(device)

# Load data
inputs = {
    ModalityType.VISION: data.load_and_transform_video_data(video_paths, device),
}

with torch.no_grad():
    embeddings = model(inputs)

print(embeddings['vision'])

## testing the clap classifier free coditional class to check it's output

In [ ]:
from audioldm_train.conditional_models  import CLAPAudioEmbeddingClassifierFreev2

model = CLAPAudioEmbeddingClassifierFreev2(
    pretrained_path="data/checkpoints/clap_htsat_tiny.pt",
    embed_mode="test",
    amodel="HTSAT-tiny",
    unconditional_prob=1
    )
# rand_px = torch.rand(1)
uncond_emb = model.get_unconditional_condition(batchsize=2)
build_emb = model.build_unconditional_emb()
uncond_emb.squeeze()
print("The unconditional shape is " , uncond_emb.shape)
print("the shape of the clap build emb is " , build_emb)

## Video to audio inferance

In [ ]:
python3 audioldm_train/infer.py --config_yaml audioldm_train/config/audioldm_original_with_imagebind/audioldm_with_imagebind.yaml --list_inference tests/captionlist/inference_test.lst --reload_from_ckpt log/latent_diffusion/audioldm_original_with_imagebind/audioldm_with_imagebind/checkpoints/checkpoint-fad-133.00-global_step=244999.ckpt